In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt


In [2]:
df_failure = pd.read_csv("failure.csv")
df_maintenance = pd.read_csv("maintenance.csv")
df_usage = pd.read_csv("usage.csv")


In [41]:
#usage preparing
df_usage_group = df_usage.groupby('Asset')
df_usage_group.min()
df_usage_group.max()

df_usage_group_max = df_usage_group.max()
df_usage_group_max.rename(columns = {"Time":"Time_max", "Use":"Use_max"}, inplace = True)
df_usage_group_max

df_usage_group_min = df_usage_group.min()
df_usage_group_min.rename(columns={"Time":"Time_min", "Use":"Use_min"}, inplace =True)
df_usage_group_min

df_usage_preparing = pd.merge(df_usage_group_min, df_usage_group_max, on="Asset")
df_usage_preparing.sort_index(axis=1)

df_usage_preparing = df_usage_preparing.sort_index(axis=1)
df_usage_preparing

,Time_max,Time_min,Use_max,Use_min
Asset,,,,
A000204,722,46,33212.761559,31449.650982
A000270,617,435,27036.768727,26378.556339
A000463,616,30,31894.002697,30451.537250
A000495,695,6,32540.252891,30851.249561
A001201,720,395,27187.900826,26136.394866
...,...,...,...,...
A998424,678,516,26923.273772,26189.203573
A998861,664,108,33306.125018,31938.933861
A998987,524,25,32224.275934,30849.498996


In [60]:
#maintenance preparing1(pivot)

df_maintenance

pd.pivot_table(df_maintenance, index=['Asset'], values='Quantity', aggfunc='sum', columns='Reason')

df_maintenance_pivot = pd.pivot_table(df_maintenance, index=['Asset'], values='Quantity', aggfunc='sum', columns='Reason')

df_maintenance_pivot

df_maintenance_preparing = df_maintenance_pivot
df_maintenance_preparing

#maintenance preparing2(Handle Missing Values)
df_maintenance_preparing = df_maintenance_preparing.fillna(0)
df_maintenance_preparing



Reason,R044,R064,R119,R193,R364,R396,R417,R446,R565,R575,R606,R707,R782,R783
Asset,,,,,,,,,,,,,,
A000204,0.0,0.0,6.0,88.0,3.0,0.0,0.0,3.0,106.0,0.0,0.0,194.0,0.0,0.0
A000270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0
A000463,0.0,0.0,3.0,96.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,229.0,0.0,6.0
A000495,0.0,0.0,4.0,123.0,13.0,0.0,0.0,39.0,394.0,0.0,0.0,145.0,0.0,0.0
A001201,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A998424,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,0.0,0.0
A998861,0.0,0.0,0.0,30.0,0.0,0.0,0.0,8.0,54.0,0.0,0.0,178.0,0.0,0.0
A998987,0.0,0.0,2391.0,50.0,1.0,0.0,0.0,5.0,128.0,0.0,0.0,100.0,0.0,0.0


In [47]:
#failure preparing

df_failure

df_failure_preparing = df_failure

df_failure_preparing

,Asset,failure_bin
0,A003146,0.0
1,A046288,0.0
2,A082667,NaN
3,A115725,1.0
4,A148200,NaN
...,...,...
1814,A845495,0.0
1815,A882703,0.0
1816,A920271,0.0
1817,A948155,0.0


In [61]:
#total data merge
tmp = pd.merge(df_usage_preparing,df_failure_preparing, on="Asset")
tmp

df = pd.merge(tmp, df_maintenance_preparing, on="Asset")
df['distance'] = df['Use_max'] - df['Use_min']
df['time_in_service'] = df['Time_max'] - df['Time_min']
df= df.drop(['Time_max', 'Time_min', 'Use_max', 'Use_min'], axis=1)
df

,Asset,failure_bin,R044,R064,R119,R193,R364,R396,R417,R446,R565,R575,R606,R707,R782,R783,distance,time_in_service
0,A000204,1.0,0.0,0.0,6.0,88.0,3.0,0.0,0.0,3.0,106.0,0.0,0.0,194.0,0.0,0.0,1763.110577,676
1,A000463,1.0,0.0,0.0,3.0,96.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,229.0,0.0,6.0,1442.465446,586
2,A000495,0.0,0.0,0.0,4.0,123.0,13.0,0.0,0.0,39.0,394.0,0.0,0.0,145.0,0.0,0.0,1689.003331,689
3,A001201,NaN,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,1051.505960,325
4,A001215,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,14.0,148.0,0.0,0.0,148.0,0.0,0.0,1555.376880,557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,A998424,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,0.0,0.0,734.070199,162
1801,A998861,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,8.0,54.0,0.0,0.0,178.0,0.0,0.0,1367.191157,556
1802,A998987,0.0,0.0,0.0,2391.0,50.0,1.0,0.0,0.0,5.0,128.0,0.0,0.0,100.0,0.0,0.0,1374.776938,499
1803,A999836,1.0,0.0,0.0,0.0,10.0,19.0,0.0,0.0,0.0,161.0,0.0,0.0,183.0,0.0,1.0,2680.698325,660


In [64]:
#prepare traingset, testset
df_train = pd.DataFrame(columns=df.columns)
df_test = pd.DataFrame(columns=df.columns)

for i in range(len(df)):
    if df['failure_bin'][i] == 1 or df['failure_bin'][i] == 0:
        df_train = df_train.append(df.iloc[i])
    else:
        df_test = df_test.append(df.iloc[i])

In [67]:
df_train

,Asset,failure_bin,R044,R064,R119,R193,R364,R396,R417,R446,R565,R575,R606,R707,R782,R783,distance,time_in_service
0,A000204,1.0,0.0,0.0,6.0,88.0,3.0,0.0,0.0,3.0,106.0,0.0,0.0,194.0,0.0,0.0,1763.110577,676
1,A000463,1.0,0.0,0.0,3.0,96.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,229.0,0.0,6.0,1442.465446,586
2,A000495,0.0,0.0,0.0,4.0,123.0,13.0,0.0,0.0,39.0,394.0,0.0,0.0,145.0,0.0,0.0,1689.003331,689
4,A001215,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,14.0,148.0,0.0,0.0,148.0,0.0,0.0,1555.376880,557
5,A002551,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,5.0,300.0,1.0,0.0,342.0,0.0,1.0,640.706739,638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1800,A998424,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,137.0,0.0,0.0,734.070199,162
1801,A998861,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,8.0,54.0,0.0,0.0,178.0,0.0,0.0,1367.191157,556
1802,A998987,0.0,0.0,0.0,2391.0,50.0,1.0,0.0,0.0,5.0,128.0,0.0,0.0,100.0,0.0,0.0,1374.776938,499
1803,A999836,1.0,0.0,0.0,0.0,10.0,19.0,0.0,0.0,0.0,161.0,0.0,0.0,183.0,0.0,1.0,2680.698325,660


In [68]:
df_test

,Asset,failure_bin,R044,R064,R119,R193,R364,R396,R417,R446,R565,R575,R606,R707,R782,R783,distance,time_in_service
3,A001201,NaN,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,1051.505960,325
9,A003641,NaN,0.0,0.0,0.0,49.0,1.0,0.0,71.0,43.0,0.0,0.0,0.0,543.0,0.0,2.0,2079.379295,556
11,A006032,NaN,0.0,0.0,0.0,103.0,0.0,0.0,114.0,0.0,248.0,0.0,0.0,240.0,0.0,13.0,704.894118,683
12,A006104,NaN,0.0,0.0,0.0,12.0,1.0,0.0,0.0,0.0,114.0,0.0,0.0,148.0,0.0,0.0,1652.533230,552
16,A010028,NaN,0.0,0.0,0.0,48.0,1.0,0.0,66.0,7.0,158.0,0.0,0.0,124.0,0.0,0.0,1619.856019,717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,A968300,NaN,5.0,0.0,0.0,35.0,72.0,0.0,0.0,90.0,57.0,0.0,0.0,192.0,0.0,0.0,1839.843670,666
1748,A975681,NaN,0.0,0.0,0.0,8.0,25.0,0.0,0.0,3.0,0.0,0.0,0.0,320.0,0.0,0.0,1938.750584,628
1761,A982449,NaN,0.0,0.0,0.0,2.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,92.0,0.0,0.0,83.443592,43
1770,A985742,NaN,0.0,0.0,0.0,7.0,90.0,0.0,97.0,69.0,0.0,0.0,0.0,244.0,0.0,0.0,1845.970647,691


In [88]:
from sklearn.model_selection import train_test_split


['Asset', 'failure_bin', 'R044', 'R064', 'R119', 'R193',
 'R364', 'R396','R417', 'R446', 'R565',
 'R575', 'R606', 'R707', 'R782', 'R783',
 'distance', 'time_in_service']

x = df_train[['R044', 'R064', 'R119', 'R193',
 'R364', 'R396','R417', 'R446', 'R565',
 'R575', 'R606', 'R707', 'R782', 'R783',
 'distance', 'time_in_service']]
y = df_train['failure_bin']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

1461
163
1461
163


In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

#training
forest = RandomForestClassifier()
forest.fit(x_train, y_train)

#predict
y_pred = forest.predict(x_test)


print('정확도 : ', metrics.accuracy_score(y_test, y_pred))

정확도 :  0.8343558282208589


In [97]:
# df_test = df_test.drop(['failure_bin'], axis=1)
# df_test = df_test.drop(['Asset'], axis=1)

df_test

,R044,R064,R119,R193,R364,R396,R417,R446,R565,R575,R606,R707,R782,R783,distance,time_in_service
3,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,1051.505960,325
9,0.0,0.0,0.0,49.0,1.0,0.0,71.0,43.0,0.0,0.0,0.0,543.0,0.0,2.0,2079.379295,556
11,0.0,0.0,0.0,103.0,0.0,0.0,114.0,0.0,248.0,0.0,0.0,240.0,0.0,13.0,704.894118,683
12,0.0,0.0,0.0,12.0,1.0,0.0,0.0,0.0,114.0,0.0,0.0,148.0,0.0,0.0,1652.533230,552
16,0.0,0.0,0.0,48.0,1.0,0.0,66.0,7.0,158.0,0.0,0.0,124.0,0.0,0.0,1619.856019,717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,5.0,0.0,0.0,35.0,72.0,0.0,0.0,90.0,57.0,0.0,0.0,192.0,0.0,0.0,1839.843670,666
1748,0.0,0.0,0.0,8.0,25.0,0.0,0.0,3.0,0.0,0.0,0.0,320.0,0.0,0.0,1938.750584,628
1761,0.0,0.0,0.0,2.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,92.0,0.0,0.0,83.443592,43
1770,0.0,0.0,0.0,7.0,90.0,0.0,97.0,69.0,0.0,0.0,0.0,244.0,0.0,0.0,1845.970647,691


In [99]:
y_pred = forest.predict(df_test)

print(y_pred)

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
